# Import Packages and Files

In [1]:
import torch.nn as nn
import dataloader as dl
import RotNet as RN
import trainer as tr

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 8000, 128)

# Train RotNet for Rotation Task

In [4]:
# initialize network
net = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']
criterion = nn.CrossEntropyLoss()

In [6]:
# train network with own implemented metric
rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net, trainloader, validloader, criterion, rot=['90', '180', '270'])

[1, 60] loss: 1.095
[1, 120] loss: 0.939
[1, 180] loss: 0.873
[1, 240] loss: 0.810
[1, 300] loss: 0.785
Epoch: 1 -> Loss: 0.33627474308
Epoch: 1 -> Validation Accuracy: 67.3875
[2, 60] loss: 0.682
[2, 120] loss: 0.621
[2, 180] loss: 0.612
[2, 240] loss: 0.612
[2, 300] loss: 0.596
Epoch: 2 -> Loss: 0.635553479195
Epoch: 2 -> Validation Accuracy: 76.04375


In [ ]:
# train network with metric from paper
rot_paper_loss_log, rot_paper_accuracy_log, rot_paper_max_accuracy, rot_paper_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [2, 3, 0, 0], 0.9, 5e-4, net, trainloader, validloader, criterion, rot=['90', '180', '270'], use_paper_metric=True)

[1, 60] loss: 1.094
[1, 120] loss: 0.947
[1, 180] loss: 0.856
[1, 240] loss: 0.839
[1, 300] loss: 0.794
Epoch: 1 -> Loss: 0.92122232914
Epoch: 1 -> Validation Accuracy: 68.4585813492
[2, 60] loss: 0.777
[2, 120] loss: 0.713
[2, 180] loss: 0.677
[2, 240] loss: 0.652
[2, 300] loss: 0.664
Epoch: 2 -> Loss: 0.476222127676
Epoch: 2 -> Validation Accuracy: 75.1705109127
[3, 60] loss: 0.536
[3, 120] loss: 0.516


In [6]:
a= 5
a -= 1
print(a)

4
